In [ ]:
import logging
logging.basicConfig(filename='logs/jupyter_log.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
logging.info('[Query Profile Main Page] Importing Libraries')
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import verticapy as vp
from IPython.display import display,clear_output, IFrame, HTML
import pickle
import re
import webbrowser
from ipyfilechooser import FileChooser
import verticapy as vp
from verticapy.performance.vertica import QueryProfilerInterface
from verticapy.connection.errors import ConnectionError
vp.set_option("print_info", False)
logging.info('[Query Profile Main Page] Successfully Imported Libraries')

In [ ]:
def reset_values(query_val = None, key_val = None, target_schema_val = None, transaction_val = None, statement_val = None, file_val = None, key_val_2 = None, target_schema_val_2 = None, transaction_statement_list_val = None):    
    if query_val:
        %store query_val
    else:
        %store -d query_val
    if key_val:
        %store key_val
    else: 
        %store -d key_val
    if target_schema_val:
        %store target_schema_val
    else:
        %store -d target_schema_val
    if transaction_val:
        %store transaction_val
    else:
        %store -d transaction_val
    if statement_val:
        %store statement_val
    else:
        %store -d statement_val
    if file_val:
        %store file_val
    else:
        %store -d file_val
    if key_val_2:
        %store key_val_2
    else: 
        %store -d key_val_2
    if target_schema_val_2:
        %store target_schema_val_2
    else:
        %store -d target_schema_val_2
    if transaction_statement_list_val:
        %store transaction_statement_list_val
    else:
        %store -d transaction_statement_list_val
    
    notebook_url = '/voila/render/ui/qprof-ui.ipynb'
    display(HTML(f"<script>window.open('{notebook_url}');window.close();</script>"))

In [ ]:
%%html
<h2> Connection Info </h2>

In [ ]:
logging.info('[Query Profile Main Page] Trying to print database connection details')
try:
    print("\033[1mDatabase\033[0m:", vp.current_connection().options['database'])
    print("\033[1mUsername\033[0m:", vp.current_connection().options['user'])
except ConnectionError:
    print("\033[91m\033[1m WARNING: No database connection found. Please connect with a database to use this tool. \033[0m\033[0m")

logging.info('[Query Profile Main Page] Done with printing database connection details ')

____

# Load

In [ ]:
button_2 = widgets.Button(description="Get Query Plan")
output_2 = widgets.Output()

key_val = None
target_schema_val = None

def on_button_clicked_2(b):
    global key_val, target_schema_val
    output_2.clear_output(wait=True)
    with output_2:
        key_val = key.value
        target_schema_val = target_schema.value
        logging.info(f"[Query Profile Main Page] For load from schema and key option, saved key value as :{key_val}, target schema as {target_schema_val}. ")
        reset_values(key_val = key_val, target_schema_val = target_schema_val)

button_2.on_click(on_button_clicked_2)


button_3 = widgets.Button(description="Get Query Plan")
file_val = None
output_3 = widgets.Output()

def on_button_clicked_3(b):
    global key_val, target_schema_val, file_val
    output_3.clear_output(wait=True)
    with output_3:
        key_val = key.value
        target_schema_val = target_schema.value
        file_val = uploader.value
        logging.info(f"[Query Profile Main Page] For load from file option, saved key value as :{key_val}, target schema as {target_schema_val}, file name as {file_val}.")
        if target_schema_val == "" or file_val == "":
            print("Error! You must provide a value for Schema and File.")
            return
        else:
            reset_values(key_val = key_val, target_schema_val = target_schema_val, file_val = file_val)

button_3.on_click(on_button_clicked_3)

In [ ]:
key = widgets.Text(description = "Key", placeholder = "Enter Key", layout=widgets.Layout(width='300px'))
key.style.description_width = '100px'
target_schema = widgets.Text(description = "Target Schema", placeholder = "Enter Schema",layout=widgets.Layout(width='300px'))
target_schema.style.description_width = '100px'
uploader = FileChooser('')

from ipyfilechooser import FileChooser


tab = widgets.Tab()
child_1 = widgets.VBox([target_schema,key,button_2, output_2])
child_2 = widgets.VBox([target_schema,key,uploader,button_3, output_3])
tab.children = [child_1, child_2]
tab.titles = ["From database", "From a file"]
logging.info('[Query Profile Main Page] Trying to display the load tab')
display(tab)
logging.info('[Query Profile Main Page] Successfully loaded')

_____

# Create

In [ ]:
button_01 = widgets.Button(description="Query Request Table", layout=widgets.Layout(width='auto'))
output_01 = widgets.Output()
display(button_01, output_01)

def on_button_clicked_01(b):
    output_01.clear_output(wait=True)
    with output_01:
        logging.info('[Query Profile Main Page] Trying to fetch the queries from query_request table')
        sql_query = "SELECT * from query_requests WHERE request_type = 'QUERY' AND request LIKE 'select%' ORDER BY request_duration_ms DESC LIMIT 6;"
        result = vp.vDataFrame(sql_query)
        logging.info('[Query Profile Main Page] Successfully fetched the queries from query_request table')
        display(HTML(result._repr_html_()))
        
logging.info('Trying to display Query Request Table button [Query Profile Main Page]')
button_01.on_click(on_button_clicked_01)
logging.info('Loaded Query Request Table button [Query Profile Main Page]')

In [ ]:
transaction = widgets.Text(description = "Transaction ID", placeholder = "Enter a Transaction ID", layout=widgets.Layout(width='300px'))
transaction.style.description_width = '100px'
statement = widgets.Text(description = "Statement ID", placeholder = "Enter a Statement ID", layout=widgets.Layout(width='300px'))
statement.style.description_width = '100px'


In [ ]:
key_2 = widgets.Text(description = "Key", placeholder = "Enter Key (Optional)", layout=widgets.Layout(width='300px'))
key_2.style.description_width = '100px'
target_schema_2 = widgets.Text(description = "Target Schema (Optional)", placeholder = "Enter Schema",layout=widgets.Layout(width='300px'))
target_schema_2.style.description_width = '150px'

In [ ]:
button_1 = widgets.Button(description="Get Query Plan")
output_1 = widgets.Output()
transaction_val = None
statement_val = None
target_schema_val_2 = None
key_val_2 = None

def on_button_clicked_1(b):
    output_1.clear_output(wait=True)
    global target_schema_val_2, key_val_2, statement_val, transaction_val
    with output_1:
        target_schema_val_2 = target_schema_2.value
        key_val_2 = key_2.value
        transaction_val = transaction.value
        statement_val = statement.value
        logging.info(f"[Query Profile Main Page] For create option, saved key value as :{key_val_2}, target schema as {target_schema_val_2}, transaction id as {transaction_val}, statement id as {statement_val}.")
        reset_values(statement_val = statement_val, transaction_val = transaction_val, target_schema_val_2 = target_schema_val_2, key_val_2 = key_val_2)

button_1.on_click(on_button_clicked_1)
create_0 = widgets.VBox([target_schema_2, key_2,transaction,statement, button_1, output_1])

In [ ]:
query = widgets.Textarea(
    placeholder='Enter a SQL Query',
    description='SQL Query:',
    disabled=False,
    layout=widgets.Layout(width='700px', height = '300px')
)

In [ ]:
button = widgets.Button(description="Get Query Plan")
output_0 = widgets.Output()
query_val = None

def on_button_clicked_0(b):
    output_0.clear_output(wait=True)
    global query_val, target_schema_val_2, key_val_2
    with output_0:
        target_schema_val_2 = target_schema_2.value
        key_val_2 = key_2.value
        query_val = query.value
        logging.info(f"""[Query Profile Main Page] For create using query option, saved key value as :{key_val_2}, target schema as {target_schema_val_2}, query as "{query_val}".""")
        reset_values(query_val = query_val, target_schema_val_2 = target_schema_val_2, key_val_2 = key_val_2)

button.on_click(on_button_clicked_0)
create_1 = widgets.VBox([target_schema_2, key_2, query, button, output_0])


In [ ]:
transaction_statement_list = widgets.Text(description = "List of transaction IDs and statement IDs", placeholder = "[(trsansac_1, statemnt_1),(trsansac_2, statemnt_2)]", layout=widgets.Layout(width='600px'))
transaction_statement_list.style.description_width = '240px'
button_4 = widgets.Button(description="Get Query Plan")
output_4 = widgets.Output()
transaction_statement_list_val = None
target_schema_val_2 = None
key_val_2 = None



def is_input_valid(input_string):
    pattern = r'\[\((\d+),(\d+)\)(,\((\d+),(\d+)\))*\]'
    return bool(re.match(pattern, input_string))

def on_button_clicked_4(b):
    output_4.clear_output(wait=True)
    global target_schema_val_2, key_val_2, transaction_statement_list_val
    with output_4:
        if not is_input_valid(transaction_statement_list.value):
            print("\033[91m\033[1m Error \033[0m: Please check the input syntax. List of tuples is expected.")
        else:
            target_schema_val_2 = target_schema_2.value
            key_val_2 = key_2.value
            transaction_statement_list_val = transaction_statement_list.value
            logging.info(f"[Query Profile Main Page] For create using multiple transaction ids and keys, saved key value as :{key_val_2}, target schema as {target_schema_val_2}, List of transactions and statements as {transaction_statement_list_val}.")
            reset_values(transaction_statement_list_val = transaction_statement_list_val, target_schema_val_2 = target_schema_val_2, key_val_2 = key_val_2)

button_4.on_click(on_button_clicked_4)
create_list_transaction_statement = widgets.VBox([target_schema_2, key_2,transaction_statement_list, button_4, output_4])

In [ ]:
tab_2 = widgets.Tab()
tab_2.children = [create_0, create_1, create_list_transaction_statement]
tab_2.titles = ["Transaction & Statement", "From SQL", "Multiple Queries"]
display(tab_2)

_____

In [ ]:
from IPython.display import HTML
logging.info(f"[Query Profile Main Page] Running script for HTML display")
display(
    HTML(
        """
<style>
.jupyter-widgets.widget-tab > .lm-TabBar .lm-TabBar-tab {
    flex: 0 1 auto
}
</style>
"""
    )
)